## Classification

In [10]:
import os
os.environ['CUDA_VISIBLE_DEVICES']='0'
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "1"

import tensorflow as tf
import numpy as np 
from prep_functions import *

np.random.seed(1234)

In [11]:
root_dir = "/home/alberto_sinigaglia/gaia"
mass_range = "CNN_low_mass"

path = f"{root_dir}/{mass_range}_train.npz"

with np.load(path, allow_pickle=False) as data:
    X = data["X"]
    y = data["y"]

In [12]:
X_train, X_val, y_train, y_val = split_train_val(X, y, val_size=0.2)

In [16]:
def make_dataset(X, y, batch_size, shuffle=True):
    '''
    This function creates a TensorFlow dataset from numpy arrays.
    '''
    X = X.astype('float32')
    X = np.reshape(X, (X.shape[0], X.shape[1], 1))
    y = y.astype('float32')
    y = np.reshape(y, (y.shape[0], 1))

    dataset = tf.data.Dataset.from_tensor_slices((X, y))

    if shuffle:
        dataset = dataset.shuffle(buffer_size=len(X))
        
    dataset = dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)
    
    return dataset

train_dataset = make_dataset(X_train, y_train, batch_size=64)
val_dataset = make_dataset(X_val, y_val, batch_size=64)

In [25]:
def basic_block(x, kernel_size, filters, name, pool_size=2):
    """Single Conv1D layer with ReLU activation + global avarage pooling"""

    x = tf.keras.layers.Conv1D(filters, kernel_size, padding='same', activation=None, name=f'{name}_conv')(x)
    x = tf.keras.layers.LeakyReLU(alpha=0.1, name=f"{name}_leaky_relu")(x)
    x = tf.keras.layers.AveragePooling1D(pool_size,  name=f'{name}_avgpooling')(x)
    return x
    

def residual_block(x, kernel_size, filters, name):
    """Two Conv1D layers + skip connection + ReLU activation"""

    shortcut = x
    x = tf.keras.layers.Conv1D(filters, kernel_size, padding='same', activation='relu', name=f'{name}_conv1')(x)
    x = tf.keras.layers.Conv1D(filters, kernel_size, padding='same', activation=None,  name=f'{name}_conv2')(x)
    x = tf.keras.layers.Add( name=f'{name}_add')([shortcut, x])
    x = tf.keras.layers.Activation('relu', name=f'{name}_relu')(x)
    return x

def residual_block_1(x, kernel_size, filters, name):

    shortcut = x 
    x = tf.keras.layers.Conv1D(filters, kernel_size, padding='same', activation=None, name=f'{name}_conv1')(x)
    x = tf.keras.layers.BatchNormalization(name=f'{name}_bn1')(x)
    x = tf.keras.layers.LeakyReLU(alpha=0.1, name=f"{name}_leaky_relu1")(x)
    x = tf.keras.layers.Conv1D(filters, kernel_size, padding='same', activation=None,  name=f'{name}_conv2')(x)
    x = tf.keras.layers.BatchNormalization(name=f'{name}_bn2')(x)
    x = tf.keras.layers.Add( name=f'{name}_add')([shortcut, x])
    x = tf.keras.layers.LeakyReLU(alpha=0.1, name=f"{name}_leaky_relu2")(x)
    return x

def residual_block_2(x, kernel_size, filters, name):

    shortcut = x 
    x = tf.keras.layers.BatchNormalization(name=f'{name}_bn1')(x)
    x = tf.keras.layers.Activation('relu', name=f'{name}_relu1')(x)
    x = tf.keras.layers.Conv1D(filters, kernel_size, padding='same', activation=None, name=f'{name}_conv1')(x)
    x = tf.keras.layers.BatchNormalization(name=f'{name}_bn2')(x)
    x = tf.keras.layers.Activation('relu', name=f'{name}_relu2')(x)
    x = tf.keras.layers.Conv1D(filters, kernel_size, padding='same', activation=None,  name=f'{name}_conv2')(x)
    x = tf.keras.layers.Add( name=f'{name}_add')([shortcut, x])
    return x

def build_model(input_length=6144, channels=1, filters=42):

    inp = tf.keras.layers.Input(shape=(input_length, channels), name='input_layer')

    x = basic_block(inp, 16, filters, name='bb_1')
    x = residual_block_1(x, 16, filters, name='rb_1')

    x = basic_block(x, 32, filters, name='bb_2')
    x = residual_block_1(x, 32, filters, name='rb_2')

    x = basic_block(x, 64, filters, name='bb_3')
    x =  residual_block_1(x, 64, filters, name='rb_3')

    x = basic_block(x, 64, filters, name='bb_4')

    x = tf.keras.layers.Flatten(name='flatten_layer')(x)

    x = tf.keras.layers.Dense(128, activation='relu', name='dl_1')(x)
    x = tf.keras.layers.Dense(64,  activation='relu', name='dl_2')(x)
    x = tf.keras.layers.Dense(64,  activation='relu', name='dl_3')(x)
    x = tf.keras.layers.Dense(32,  activation='relu', name='dl_4')(x)

    out = tf.keras.layers.Dense(1, activation='sigmoid', name='output_layer')(x)

    model =  tf.keras.Model(inp, out, name='classification_model')
    return model

model = build_model()
model.summary()

/home/alberto_sinigaglia/anaconda3/envs/gaia/lib/python3.10/site-packages/keras/src/layers/activations/leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


Model: "classification_model"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 6144, 1)   │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bb_1_conv (Conv1D)  │ (None, 6144, 42)  │        714 │ input_layer[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bb_1_leaky_relu     │ (None, 6144, 42)  │          0 │ bb_1_conv[0][0]   │
│ (LeakyReLU)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bb_1_avgpooling     │ (None, 3072, 42)  │          0 │ bb_1_leaky_relu[… │
│ (AveragePooling1D)  │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ rb_1_conv1 (Conv1D) │ (None, 3072, 42)  │     28,266 │ bb_1_avgpooling[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ rb_1_bn1            │ (None, 3072, 42)  │        168 │ rb_1_conv1[0][0]  │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ rb_1_leaky_relu1    │ (None, 3072, 42)  │          0 │ rb_1_bn1[0][0]    │
│ (LeakyReLU)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ rb_1_conv2 (Conv1D) │ (None, 3072, 42)  │     28,266 │ rb_1_leaky_relu1… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ rb_1_bn2            │ (None, 3072, 42)  │        168 │ rb_1_conv2[0][0]  │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ rb_1_add (Add)      │ (None, 3072, 42)  │          0 │ bb_1_avgpooling[… │
│                     │                   │            │ rb_1_bn2[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ rb_1_leaky_relu2    │ (None, 3072, 42)  │          0 │ rb_1_add[0][0]    │
│ (LeakyReLU)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bb_2_conv (Conv1D)  │ (None, 3072, 42)  │     56,490 │ rb_1_leaky_relu2… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bb_2_leaky_relu     │ (None, 3072, 42)  │          0 │ bb_2_conv[0][0]   │
│ (LeakyReLU)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bb_2_avgpooling     │ (None, 1536, 42)  │          0 │ bb_2_leaky_relu[… │
│ (AveragePooling1D)  │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ rb_2_conv1 (Conv1D) │ (None, 1536, 42)  │     56,490 │ bb_2_avgpooling[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ rb_2_bn1            │ (None, 1536, 42)  │        168 │ rb_2_conv1[0][0]  │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ rb_2_leaky_relu1    │ (None, 1536, 42)  │          0 │ rb_2_bn1[0][0]    │
│ (LeakyReLU)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ rb_2_conv2 (Conv1D) │ (None, 1536, 42)  │     56,490 │ rb_2_leaky_relu1… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ rb_2_bn2            │ (None, 1536, 42)  │        168 │ rb_2_conv2[0][0]

 Total params: 2,758,517 (10.52 MB)

 Trainable params: 2,758,013 (10.52 MB)

 Non-trainable params: 504 (1.97 KB)

In [30]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3), 
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=False), 
              metrics=[tf.keras.metrics.BinaryAccuracy(name="acc"), 
                       tf.keras.metrics.TruePositives(name="true_positives"),
                       tf.keras.metrics.FalsePositives(name="false_positives"),
                       tf.keras.metrics.Recall(name="recall")])

callbacks = [
    tf.keras.callbacks.ModelCheckpoint(
        filepath=f"{mass_range}_best_model.keras", 
        monitor="recall",
        mode="max",
        save_best_only=True,
        verbose=0,
    ),
    tf.keras.callbacks.CSVLogger(
        filename=f"training_log_{mass_range}.csv",
        append=False,
    )
]

In [ ]:
'''
    tf.keras.callbacks.EarlyStopping(
        monitor="val_recall",
        mode="max",
        patience=8,           # stop if no improvement for 8 epochs
        min_delta=1e-4,       # ignore tiny bumps
        restore_best_weights=True,
        verbose=0,
    )
'''

In [ ]:
history = model.fit(
    x=train_dataset,
    validation_data=val_dataset,
    epochs=200,
    callbacks=callbacks,
    verbose=1,
)

Epoch 1/200
182/182 ━━━━━━━━━━━━━━━━━━━━ 19s 65ms/step - acc: 0.9736 - false_positives: 132.0000 - loss: 0.0761 - recall: 0.9699 - true_positives: 5633.0000 - val_acc: 0.9487 - val_false_positives: 43.0000 - val_loss: 0.1530 - val_recall: 0.9270 - val_true_positives: 1346.0000
Epoch 2/200
182/182 ━━━━━━━━━━━━━━━━━━━━ 6s 33ms/step - acc: 0.9771 - false_positives: 117.0000 - loss: 0.0647 - recall: 0.9743 - true_positives: 5659.0000 - val_acc: 0.9463 - val_false_positives: 101.0000 - val_loss: 0.1628 - val_recall: 0.9621 - val_true_positives: 1397.0000
Epoch 3/200
182/182 ━━━━━━━━━━━━━━━━━━━━ 6s 33ms/step - acc: 0.9811 - false_positives: 93.0000 - loss: 0.0564 - recall: 0.9781 - true_positives: 5681.0000 - val_acc: 0.9421 - val_false_positives: 89.0000 - val_loss: 0.1940 - val_recall: 0.9456 - val_true_positives: 1373.0000
Epoch 4/200
182/182 ━━━━━━━━━━━━━━━━━━━━ 6s 32ms/step - acc: 0.9789 - false_positives: 110.0000 - loss: 0.0588 - recall: 0.9768 - true_positives: 5673.0000 - val_acc: 0

: 